install pytorch on GCP: https://discuss.pytorch.org/t/deploy-pretrained-pytorch-model-on-google-cloud/14664

In [0]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.0.0
True


In [0]:
torch.backends.cudnn.version()

7401

In [0]:
'blabla'+'_a'*True +'_b'*False

'blabla_a'

# Data

In [0]:
import numpy as np
import pickle
from PIL import Image
import time
import shutil
import random
import argparse
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau


from __future__ import print_function, division
import pickle
import torch
import torch.nn as nn
import glob
from PIL import Image
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
#import torchvision
#import dataloader # LISA: dataloader is a folder of the github repo of two-stream conv fusion. I have copied all the necessary function above.
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage
import re
import time
import os
import copy

In [0]:
# parameters for temporal chuking:
temporal_chunking = True 
if temporal_chunking:
    L = 6 
else:
    L = 40 #this can't be modified since there is only 40 frames per video !

####  two-stream-fusion-for-action-recognition-in-videos/dataloader/split_train_test_video.py


In [0]:
a = '/data/vision/billf/donglai-lib/Data/High/Classify/UCF101/bflowb_orig/BoxingPunchingBag/v_BoxingPunchingBag_g02_c02 280 1'
a.split('/',11)[-1].split(' ')[-1]

'1'

In [0]:
class UCF101_splitter():
    def __init__(self, path, split):
        self.path = path # /train_val_split ?
        self.split = split

    
    
    def split_video(self):
        train_video = self.file2_dic(self.path+'train01_cnn_ta_flow_orig_fb.txt')
        test_video = self.file2_dic(self.path+'test01_cnn_ta_flow_orig_fb.txt')
        print('==> (Training video, Validation video):(', len(train_video),len(test_video),')')
        self.train_video = self.make_dict(train_video)
        self.test_video = self.make_dict(test_video)

        return self.train_video, self.test_video

    def file2_dic(self, fname):
        with open(fname) as f:
            content = f.readlines()
            content = [x.strip('\r\n') for x in content]
        f.close()
        dic = {}
        with open('video_labels.pickle', 'rb') as handle:
            video_labels = pickle.load(handle)
        handle.close()
        count = 0
        for line in content:
            video_name = line.split('/',6)[-2]
            video_class = line.split('/',6)[-1].split(' ')[0]

            #print('video_name',video_name)
            #print('video_class: ', video_class)
            
            video_name_class = video_name+'_' +video_class #b'*(video_class == 0)+'_f'*(video_class == 1)
            #print('video_name_class: ',video_name_class)

            #print('video_labels[video_name_class]: ', video_labels[video_name_class])
            try:
                #print('video_name_class[2:]: ', video_name_class[:2])
                #print('video_name_class: ',video_name_class)
                dic[video_name_class[2:]] = video_labels[video_name_class]
                
            except: 
                count+=1
        print('they are '+str(count)+' missing videos')
                
        return(dic)
    
    def file2_dic_(self,fname): 
        # here we keep the same split test train as in th text file for action recognition...
        # do we need to keep both backward and forward for the training ?
        with open(fname) as f:
            content = f.readlines()
            content = [x.strip('\r\n') for x in content]
#            print('file2_dic / content[:5]', content[:5])
        f.close()
        with open('video_labels.pickle', 'rb') as handle:
            video_labels = pickle.load(handle)
        handle.close()
        dic={}
        count = 0
        
        for line in content:
            #print line
            video = line.split('/',1)[1].split(' ',1)[0]
            key = video.split('_',1)[1].split('.',1)[0]
            #print('key+\'_b\'',key+'_b')
            #label = self.action_label[line.split('/')[0]]   
            #print('v'+key+'_b')
            arrow = np.random.randint(2)
            if arrow == 0:
                try:
                    dic[key+'_b'] = video_labels['v_'+key+'_b']
                #print('dic[key+\'_b\'],dic[key+\'_f\']',dic[key+'_b'],dic[key+'_f'])
                except:
                    count+=1
            else:
                try:
                    dic[key+'_f'] = video_labels['v_'+key+'_f']
                except:
                    count+=1
        print('they are '+str(count)+' missing videos')
            
            #dic[key] = int(label)
            #print key,label
        return dic

    def make_dict(self,dic):
        dic2 = {}
        count = 0
        for video in dic:
            #if video.split('_')[0]!='HandStandPushups': 
                # LISA: [correction] because in the frame_count.pickle file, there is no any "HandStandPushups" video
            dic2[video] = float(dic[video]) # LISA: [correction] it was written dic2[videoname] = dic[video] but videoname is not assigned...
#                if count <5:
#                    print(video, dic[video])
#                count +=1
        return dic2

In [0]:
with open('video_labels.pickle', 'rb') as handle:
            video_labels = pickle.load(handle)
handle.close()

for k in list(video_labels.keys())[:5]:
    print(k,video_labels[k])

v_GolfSwing_g21_c02_b 0
v_PlayingDaf_g13_c02_b 0
v_TrampolineJumping_g23_c04_b 0
v_Nunchucks_g22_c02_b 0
v_Fencing_g02_c04_f 1


####  two-stream-fusion-for-action-recognition-in-videos/dataloader/temp_loader.py


In [0]:
class motion_dataset(Dataset):  
    def __init__(self, dic, in_channel, root_dir, mode, transform=None):
        self.keys=dic.keys()
        self.values=dic.values()
        self.root_dir = root_dir
        self.transform = transform
        self.mode=mode
        self.in_channel = in_channel # be careful we only have 40 frames...
        self.img_rows=224
        self.img_cols=224

    def stackopf(self):
        name = 'v_'+self.video
        u = self.root_dir+ name + '/x'
        v = self.root_dir+ name+ '/y'
        #print(u,v)
        
        flow = torch.FloatTensor(2*self.in_channel,self.img_rows,self.img_cols)
        i = int(self.clips_idx)
#        print(i)


        for j in range(self.in_channel):        
            idx = i + j #LISA: i + j since in_channel = 50 and num_frames = O(50)
            idx = str(idx)
            #frame_idx = 'frame'+ idx.zfill(6)
            frame_idx = idx.zfill(4)
            h_image = u +'/' + frame_idx +'.jpg'
            v_image = v +'/' + frame_idx +'.jpg'
            
            imgH=(Image.open(h_image))
            imgV=(Image.open(v_image))

            H = self.transform(imgH)
            V = self.transform(imgV)

            
            flow[2*(j-1),:,:] = H
            flow[2*(j-1)+1,:,:] = V      
            imgH.close()
            imgV.close()  
        return flow

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):  ### LISA: what is idx ?? is it a list ?
        #print ('mode:',self.mode,'calling Dataset:__getitem__ @ idx=%d'%idx)
        
        if self.mode == 'train' or 'val':
            ########
            # LISA #
            ########
            # here is the call to the videos
            # question, when do we pass the forward and backward videos ? can we pass it throw the same batch?
            
            #self.video, nb_clips = list(self.keys)[idx].split('-') #LISA: [correction] self.keys[idx] doesn't work
            self.video = list(self.keys)[idx]
            self.clips_idx = 1
        else:
            raise ValueError('There are only train and val mode')

        #print('idx: ', idx, type(idx))
        
        label = float(list(self.values)[idx]) #LISA: [correction] self.values[idx] doesn't work
        #print('label: ',label)
        #print('int(label): ',int(label))
        #label = float(label) 
        #print('label: ',label, type(label))
        data = self.stackopf()

        if self.mode == 'train' or 'val':
            sample = (data,label)
        else:
            raise ValueError('There are only train and val mode')
        return sample

In [0]:
class Motion_DataLoader():
    def __init__(self, BATCH_SIZE, num_workers, in_channel,  path, ucf_list, ucf_split,train_transform,val_transform):

        self.BATCH_SIZE=BATCH_SIZE
        self.num_workers = num_workers
        self.frame_count={}
        self.in_channel = in_channel
        self.data_path=path
        self.path_ucf = ucf_list
        self.train_transform = train_transform
        self.val_transform = val_transform
        # split the training and testing videos
        splitter = UCF101_splitter(path=ucf_list,split=ucf_split)
        self.train_video, self.test_video = splitter.split_video()
        
    def load_frame_count(self):
        #loading the frame count for videos
        with open(self.path_ucf+'/frame_count.pickle','rb') as file:
            dic_frame = pickle.load(file)
        file.close()
#         print("Now in loadframe_count ")

        for line in dic_frame : # LISA: removing the v_ and the .avi
            videoname = line.split('_',1)[1].split('.',1)[0]
            n,g = videoname.split('_',1)
            self.frame_count[videoname]=dic_frame[line]

    def run(self):
        #self.load_frame_count()
        #self.get_training_dic()
        #self.val_sample()
        train_loader = self.train()
        val_loader = self.val()

        return train_loader, val_loader, self.test_video
            
    def train(self):
        training_set = motion_dataset(dic=self.train_video, in_channel=self.in_channel, root_dir=self.data_path,
        #training_set = motion_dataset(dic=self.dic_video_train, in_channel=self.in_channel, root_dir=self.data_path,
            mode='train',
            transform = self.train_transform)
        #transforms.Compose([
        #    transforms.Resize([224,224]),
        #    transforms.ToTensor(),
        #    ])
        print('==> Training data :',len(training_set),' videos',training_set[1][0].size())
        
        train_loader = DataLoader(
            dataset=training_set, 
            batch_size=self.BATCH_SIZE,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True
            )

        return train_loader

    def val(self):
        validation_set = motion_dataset(dic= self.test_video, in_channel=self.in_channel, root_dir=self.data_path ,
        #validation_set = motion_dataset(dic= self.dic_test_idx, in_channel=self.in_channel, root_dir=self.data_path ,
            mode ='val',
            transform =self.val_transform)
         #transforms.Compose([
            #transforms.Resize([224,224]),
            #transforms.ToTensor(),
            #])
        print('==> Validation data :',len(validation_set),' frames',validation_set[1][0].size())
        #print validation_set[1]

        val_loader = DataLoader(
            dataset=validation_set, 
            batch_size=self.BATCH_SIZE, 
            shuffle=False,
            num_workers=self.num_workers)

        return val_loader


# Two-Stream convolutional fusion

## plots: 
> ### confusion matrix, images..

In [0]:
# for computing confusion matrix
def compute_confusion_matrix(y_actu, y_pred, class_names):
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_actu, y_pred)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    #uncomment below lines if you want to plot non-normalized matrix
    #plt.figure()
    #plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure(figsize=(20,20))
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

    plt.show()

    
#for plotting confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
#function for showing the inputs at the time of loading of data
def show_images(inputs,label):
    for i in range(inputs.shape[0]):
        a = inputs[i:i+1,:]
        a = a.view([int(inputs.size(1)/3),3,224,224])

        out = torchvision.utils.make_grid(a)
        out = out.numpy().transpose((1, 2, 0))
        out = [0.229, 0.224, 0.225] * out + [0.485, 0.456, 0.406]
        out = np.clip(out, 0, 1)
        plt.imshow(out)
        plt.pause(0.001) 
        print("Patches of RGB frames for batch %d for class %d" % (i,label[i]+1))


## functions to adapt the backbone model 

In [0]:
#function for getting weights of first layer of pre-trained model and first layer of model defined on scratch
def get_weights(trained_model, untrained_model):
    i = 0
    for param in untrained_model.parameters():
        if(i == 0):
            untrained_weights = param.data
        i += 1
        
    i = 0
    for param in trained_model.parameters():
        if(i == 0):
            trained_weights = param.data
        i += 1 
        
    return [trained_weights, untrained_weights]
    

#function for appending the trained model weights to the first layer of model defined on scratch
def concat_trained_weights(trained_weights, untrained_weights):
    #averaging of trained filter weights
    temp           = torch.sum(trained_weights[:,:], dim= 1)[0] / 3

    filter_dim     = untrained_weights.size(1)
    filter_weights = temp.view(1,3,3)
    for i in range(filter_dim-1):
        filter_weights = torch.cat((filter_weights, temp.view(1,3,3)), dim=0)

    for i in range(untrained_weights.size(0)):
        untrained_weights[i,:,:,:] = filter_weights
    
    return untrained_weights


## training the model

In [0]:
'''
Function for training of model
This function can be modified accordingly to include validation set
'''
    
def train_model(model, criterion, optimizer,scheduler, num_epochs, phase, temporal_chunking = False, L = 6): 
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        #for phase in ['train']:
        j = 0
        scheduler.step()
        model.train()

        epoch_loss = 0.0
        correct_samples = 0
        batch_num = 0
        # Iterate over data.
        for i_data,data in enumerate(fullloader[phase]):  ## LISA: fullloader is a global variable created when "loading the train and test data"
            #print('loading the data for batch: ', batch_num)
            #print('i_data,data: ',i_data,data)
            batch_num+=1
            temp_data, labels = data 

            if use_gpu:
                temp_data = Variable(temp_data.cuda())
                labels    = Variable(labels.cuda())
            else:
                temp_data = Variable(temp_data)
                labels    = Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            #temporal_chunking to fill before !
            #outputs = model(temp_data, temporal_chunking, tau, )
            outputs = model(temp_data, temporal_chunking = temporal_chunking, L = L)
            #print('outputs: ', outputs.size())
            #print('th: ', torch.Tensor([0.5]*fullloader[phase].batch_size))
            #print('(Variable(torch.Tensor([0.5]*fullloader[phase].batch_size)) > outputs.cpu()): ', (Variable(torch.Tensor([0.5]*fullloader[phase].batch_size)) > outputs.cpu()))
                
            th = Variable(torch.Tensor([0.5]*outputs.size(0)))
            outputs_variable = Variable(outputs)
                #print('outputs_variable: ', outputs_variable)
                #print('th: ', th)
            preds = (th < outputs_variable.cpu()).float() * 1
            #print('preds: ',preds)
                #_,preds = torch.max(outputs, 1)  
                #print('preds: ',preds)
                #pred = outputs.data.max(1, keepdim=True)[1]
            del temp_data
                
            # backward and optimize for training mode
            loss = criterion(outputs.float(), labels.float())
                
            loss.backward()
            optimizer.step()
            print('loss for batch {0} in epoch {1}: {2}'.format(batch_num, epoch,loss.item()))
            # finding epoch losses and accuracies for fusion
            epoch_loss += loss.item() #* spat_data.size(0)

            #correct_samples += torch.sum(preds == labels.data)
                
            correct_samples += preds.eq(labels.float().cpu().data.view_as(preds)).cpu().sum()
            del labels
            #print("Correct Samples = ", correct_samples)
                                         
        phase_size = len(fullloader[phase])*fullloader[phase].batch_size
#             print("phase size = ", phase_size)
        epoch_acc = correct_samples / phase_size 
        epoch_loss = epoch_loss / len(fullloader[phase])
        
        if phase == 'train':
            if epoch % 10 == 0: # 
                torch.save(model.state_dict(), 'models_pth/training_epoch_{0}.pth'.format(epoch))

            #print("=================== Training mode ====================")
        #print('Loss: {:.4f} '.format(
                #epoch_loss))
        
        print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch_loss, correct_samples, phase_size,
        100. * correct_samples / phase_size))
        #batches_done = fullloader[phase].batch_size*(epoch - 1) + outputs.size(0)
        #batches_left = fullloader[phase]*(n_epochs - epoch) + num_epochs - self.batch 
        #sys.stdout.write('ETA: %s' % (datetime.timedelta(seconds=batches_left*self.mean_period/batches_done)))
            
        acc = 100 * correct_samples /phase_size
        
    
        if acc > best_acc:
            best_acc = acc
            best_epoch = epoch 
            print('*', end='')
            # Save model onto disk.
                
            if phase == 'train':
  
                torch.save(model.state_dict(), 'models_pth/best_epoch.pth')
    
            print('Best at epoch %d, test accuaray %f' % (best_epoch, best_acc))

    time_elapsed = time.time() - since
    print('Training completed in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best at epoch %d, test accuaray %f' % (best_epoch, best_acc))
    return model

## loading the train and test data

In [0]:
#loading the train and test data
'''
change here for loading data for spatial loader, temporal loader

'''
#############################
# ADAPTATION FOR TCAM MODEL #
#############################

# LISA: perform only the temporal loading
# LISA: be careful with the data transformation: data augmentation but also artificial cues removing
data_loader = Motion_DataLoader(BATCH_SIZE=128, num_workers=1, in_channel = 40, #LISA: why ??
    #def __init__(self, BATCH_SIZE, num_workers, in_channel,  path, ucf_list, ucf_split)
#data_loader = spatio_temporal_dataloader(BATCH_SIZE=16, num_workers=1, in_channel = 50, #LISA: why ??
                                #spatial_path='/home/lisabedin/test/jpegs_256/',    
                                path='/home/lisabedin/dataset_AoT/',
                                ucf_list='/home/lisabedin/',# train_val_split #ucf_list
                                ucf_split='02',
                                train_transform = transforms.Compose([
                                                   transforms.RandomCrop(224),
                                                   transforms.RandomHorizontalFlip(),
                                                   transforms.ToTensor(),
                                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                        std=[0.229, 0.224, 0.225])
                                               ]),
                                val_transform = transforms.Compose([
                                                     transforms.Resize([224,224]),
                                                     transforms.ToTensor(),
                                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])
                                                 ]))
train_loader, test_loader, test_video = data_loader.run()
'''
appending train-loader and test loader for training the model
'''
fullloader = {}
fullloader['train'] = train_loader
fullloader['val'] = test_loader
del train_loader
del test_loader

'''
loading classes name for computing confusion matrix
'''
temp = []
with open("/home/lisabedin/UCF_list/classInd.txt", "r") as f:
    for line in f:
        abc = line.split(" ")[1]
        #print(abc)
        temp.append(abc.rstrip("\n"))
f.close()
#print(temp)

they are 0 missing videos
they are 0 missing videos
==> (Training video, Validation video):( 14848 6364 )
==> Training data : 14848  videos torch.Size([80, 224, 224])
==> Validation data : 6364  frames torch.Size([80, 224, 224])


## Model

In [0]:
'''
Defining a model model which will do convolution fusion of both stream and 3D pooling 
'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## LISA: spat_model and temp_model are global variable defined in the cell below 
        
        self.temp_feature = temp_model.features #feature_size = Nx512x7x7
        
        self.pooling_temporal_chunks = nn.MaxPool3d(5,5)
        
        self.conv2d1 = nn.Sequential(
            nn.Conv2d(512, 1024,kernel_size=3, padding=1), #853
            nn.BatchNorm2d(1024,1e-3),
            nn.ReLU(True),
            nn.Dropout(0.5))
        
        self.conv2d2 = nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024,1e-3),
            nn.ReLU(True),
            nn.Dropout(0.5))
        
        self.CAM_avg_pool = nn.AvgPool2d(14,14)#.cuda()
        
        self.classifier = nn.Sequential(nn.Linear(1024,1), # LISA: batch_size be careful !!
            nn.Sigmoid())#.cuda()
        
    def forward(self, x, temporal_chunking = False, L = 6, T = 5):
        #print('x input: ', x.size())
        
        # temporal chunking:
        if temporal_chunking:
            stride = int(np.random.randint(L))
            #print('stride',stride)
            starting_frame = int(np.random.randint(int(x.size(1)/2))) #x.size(1) = 80 since there is the flow according x and the flow acording y
        
            if starting_frame < L/2:
                starting_frame = L/2 # this is the starting frame according one direction !!
                
            elif starting_frame + (T-1) * stride +2 >= x.size(1)/2 - L/2: ### T sampling optical flow stacks!!
                starting_frame = x.size(1)/2 - L/2 - (T-1) * stride -2
            
            current_frame = int(2*starting_frame)
            #print('starting_frame, current_frame: ', starting_frame,current_frame)
            
            #print('current frame 0: ',current_frame)
            current_tensor = x[:,current_frame-L:current_frame+L+2,:,:]
            #print('current tensor 0: ', current_tensor.size())
            
            y = self.temp_feature(current_tensor)
            #print('temp stride 0: ', y.size())
            for k in range(T-1):
                current_frame = int(current_frame + 2*stride)
                #print('current frame {0}: '.format(k+1),current_frame)
                current_tensor = x[:,current_frame-L:current_frame+L+2,:,:]
                #print('current tensor {0}: '.format(k+1), current_tensor.size())
                #print('current_frame-L:current_frame+L+2: ',current_frame-L,current_frame+L+2)
                
                y = torch.cat((y,self.temp_feature(current_tensor)), dim= 1)
                #print('temp cat '+str(k+1)+': ', y.size())
                
            y = self.pooling_temporal_chunks(y)
            #print('pooling_temporal_chunks: ', y.size())
        else: 
        
            y = self.temp_feature(x)
            #print('self.temp_feature(x): ',x.size())
        
        # CAM model for classification
        y = self.conv2d1(y)
        #print('self.conv2d1(x): ', x.size())
        
        y = self.conv2d2(y)
        #print('self.conv2d2(x): ', x.size())

        y = self.CAM_avg_pool(y)
        #print('avg: ', x.size())
          
        y = y.view(-1,1024)
        #print('x.view(-1,1024): ',x.size())
        
        y = self.classifier(y)
        #print('self.classifier(x): ', x.size())
        y = y.view(y.size(0))
        #print('view: ', x.size())
        #print('x: ',x)
        #print('type(x): ', type(x))
        return y

## Initialization of the model: 

> ### pretraining and adapatation



In [0]:
'''
using pre-trained weights on ImageNet for both streams
'''
#LISA: be careful, run this cell only one time otherwise you'll get GPU memory errors
since = time.time()
#spat_model = models.vgg16(pretrained=False) #LISA: change with pretrained=True (but longer)
temp_model = models.vgg16(pretrained=False)
print('vgg16 loaded')

time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

vgg16 loaded
Training completed in 0m 3s


In [0]:
'''
first average the weight value across the RGB channels 
and replicate this average by the channel number of model
'''

#changing input filter dimension of temporal model 
feat_     = list(temp_model.features.children())[:-1]# LISA: as in the T-CAM model removing all the layers after 
class_  = list(temp_model.classifier.children())
if temporal_chunking:
    feat_[0]  = nn.Conv2d(2*L+2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) 
else:
    feat_[0]  = nn.Conv2d(2*L, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # 80= 2*40 channels
class_[6] = nn.Linear(in_features=4096, out_features=101, bias=True)

#temporary model 
abc = nn.Sequential(*feat_) 

[trained_weights, untrained_weights] = get_weights(temp_model, abc)
trained_weights                      = concat_trained_weights(trained_weights, untrained_weights)

i = 0
for param in abc.parameters():
    if(i == 0):
        param.data = trained_weights
    i += 1 
    
temp_model.features = abc
temp_model.classifier = nn.Sequential(*class_)

## Main

In [0]:
'''
if system has cuda 
'''
use_gpu = torch.cuda.is_available()
print(torch.cuda.memory_allocated())#device=None

model   = Net()

if use_gpu:
    #model = model.cuda()
    model = torch.nn.DataParallel(model,device_ids=[0,1,2,3]).cuda()
    cudnn.benchmark=True


criterion = nn.BCELoss() #CrossEntropyLoss()


optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 10 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


model = train_model(model, criterion, optimizer,scheduler, num_epochs=430, phase='train', temporal_chunking = temporal_chunking,L=L)

0
Epoch 0/429
----------
outputs:  torch.Size([128])
loss for batch 1 in epoch 0: 0.6973584294319153
outputs:  torch.Size([128])
loss for batch 2 in epoch 0: 0.6953518986701965
outputs:  torch.Size([128])
loss for batch 3 in epoch 0: 0.6941211223602295
outputs:  torch.Size([128])
loss for batch 4 in epoch 0: 0.6949287056922913
outputs:  torch.Size([128])
loss for batch 5 in epoch 0: 0.6942180395126343


In [0]:
# Test

In [0]:
state_dict = torch.load('models_pth/best_epoch.pth')
model = Net()
model.load_state_dict(state_dict)
#model.eval()

model.cuda()

model = train_model(model, criterion, optimizer,scheduler, num_epochs=1, phase='val')